In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.stats import norm

In [2]:
N = norm.cdf

def BS_CALL(S, K, T, r, q, sigma):
    if sigma * np.sqrt(T) != 0.0:
        d1 = (np.log(S/K) + (r - q + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return S * np.exp(-q*T) * N(d1) - K * np.exp(-r*T) * N(d2)
    else:
        return S * np.exp(-q*T)

def BS_PUT(S, K, T, r, q, sigma):
    if sigma * np.sqrt(T) != 0.0:
        d1 = (np.log(S/K) + (r - q + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return K * np.exp(-r*T) * N(-d2) - S * np.exp(-q*T) * N(-d1)
    else:
        return K * np.exp(-r*T)

# dataframe is each row
def BS(dataframe):
    S = dataframe['adj_close']
    K = dataframe['strike']
    T = dataframe['expiration_days'] / 365
    r = dataframe['disc_rate']
    q = dataframe['div_yield']
    sigma = dataframe['implied_vol']
    CP = dataframe['call/put']
    if CP == 'C':
        return BS_CALL(S, K, T, r, q, sigma)
    elif CP == 'P':
        return BS_PUT(S, K, T, r, q, sigma)
    else:
        return None

def BS_vol(dataframe):
    S = dataframe['adj_close']
    K = dataframe['strike']
    T = dataframe['expiration_days'] / 365
    r = dataframe['disc_rate']
    q = dataframe['div_yield']
    price = (dataframe['best_bid'] + dataframe['best_offer']) / 2
    CP = dataframe['call/put']
    error = 0.005
    left = 0.0
    right = 50.0
    iterations = 1000
    t = 0
    m = (left + right) / 2.0
    if CP == 'C':
        diff = BS_CALL(S, K, T, r, q, m) - price
        while abs(diff) > error and t < iterations:
            if diff > 0:
                right = m
            else:
                left = m
            m = (left + right) / 2.0
            diff = BS_CALL(S, K, T, r, q, m) - price
            t += 1
        return m
    elif CP == 'P':
        diff = BS_PUT(S, K, T, r, q, m) - price
        while abs(diff) > error and t < iterations:
            if diff > 0:
                right = m
            else:
                left = m
            m = (left + right) / 2.0
            diff = BS_PUT(S, K, T, r, q, m) - price
            t += 1
        return m
    else:
        return None

In [3]:
df = pd.read_csv('example_options_history.csv')
df = df.drop_duplicates()

In [4]:
df_curr = df[df['date'] == '2023-06-30'][['adj_close', 'div_yield', 'strike', 'expiration_days', 'call/put', 'best_bid', 'best_offer',
       'volume', 'implied_vol', 'delta', 'disc_rate', 'expiration']]

In [5]:
df_curr['BS_price'] = df_curr.apply(BS, axis=1)

In [6]:
df_curr['BS_vol'] = df_curr.apply(BS_vol, axis=1)

In [7]:
df_curr

,adj_close,div_yield,strike,expiration_days,call/put,best_bid,best_offer,volume,implied_vol,delta,disc_rate,expiration,BS_price,BS_vol
327248,193.97,0.004795,125.0,567,P,2.40,3.10,100,0.316518,-0.070095,0.052078,2025-01-17,2.640179,0.320053
327249,193.97,0.004795,120.0,567,P,1.92,2.80,3,0.324052,-0.060203,0.052078,2025-01-17,2.271705,0.327301
327250,193.97,0.004795,115.0,567,P,1.33,2.69,0,0.331537,-0.051349,0.052078,2025-01-17,1.938895,0.334549
327251,193.97,0.004795,110.0,567,P,0.87,2.77,348,0.344443,-0.045381,0.052078,2025-01-17,1.759705,0.347137
327252,193.97,0.004795,105.0,567,P,0.88,2.14,0,0.351101,-0.037921,0.052078,2025-01-17,1.462841,0.353622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329217,193.97,0.004795,50.0,14,C,143.60,145.70,0,NaN,NaN,0.047446,2023-07-14,NaN,3.546143
329218,193.97,0.004795,60.0,14,C,133.45,135.85,0,NaN,NaN,0.047446,2023-07-14,NaN,3.076172
329219,193.97,0.004795,75.0,14,C,118.55,120.85,0,2.518991,0.985270,0.047446,2023-07-14,119.665022,2.545166
329220,193.97,0.004795,70.0,14,C,123.55,125.80,0,2.675275,0.986501,0.047446,2023-07-14,124.639940,2.697754
